# Primeiro dia - Introdução: Orquestração, `pyepics`, `ophyd` and `bluesky`

##### Setup

For those examples to work, you'll need to run the two soft IOCs these codes interact with. One is a python-softioc based IOC, while the other is an epics-base simulated motor IOC, using SIRIUS's `motor_sim_epics_ioc` docker container (accessible in the intranet).

Both of these IOCs are on the `../iocs/` directory. The tested way to run those is via the `iocs` script, like so:

```bash
cd .../iocs/
iocs start --local --anyuser
```

In [ ]:
def timed_function(print_timing):
    def __wrapper__(fn):
        import epics
        import time
        import functools
        @functools.wraps(fn)
        def __inner__(*args, **kwargs):
            epics.pv._PVcache_.clear()
            epics.ca.clear_cache()

            _t = time.perf_counter()
            ret = fn(*args, **kwargs)
            _dt = time.perf_counter() - _t
            if print_timing:
                print(f"Function '{fn.__name__}' executed in {_dt:.4f}s.")
            return ret
        return __inner__
    return __wrapper__

### Orquestração de experimentos nas linhas de luz



![Diagrama do esquema EPICS / Orquestração no SIRIUS](./images/orquestracao_de_experimentos.png)

### Estado atual: `pyepics`

#### Estudo de caso: Orquestração detector + motor

![Example setup diagram, with a punctual detector and a pace motor moving the sample along the y axis.](./images/example_setup_diagram.png)

|Prefixo|Tipo|Descrição|
|-------|----|---------|
|TEST:DETECTOR:AcquisitionNumber|Setpoint|Sobrescreve o número de aquisições feitas até o momento.|
|TEST:DETECTOR:AcquisitionNumber_RBV|Readback|Indica o número de aquisições realizadas até o momento.|
|TEST:DETECTOR:Trigger|Setpoint|Adquire a leitura atual do detector e salva os dados internamente|
|TEST:DETECTOR:Trigger_RBV|Readback|Indica se uma nova leitura está sendo adquirida (1) ou se já foi adquirida (0).|
|TEST:DETECTOR:Data|Setpoint|Sobrescreve a leitura mais recente com inputs manuais.|
|TEST:DETECTOR:Data_RBV|Readback|Mostra a valor da última leitura do detector.|

|Prefixo|Tipo|Descrição|
|-------|----|---------|
|TEST:MOTORS:m1|Motor|Eixo X do estágio da amostra|
|TEST:MOTORS:m2|Motor|Eixo Y do estágio da amostra|
|TEST:MOTORS:m3|Motor|Eixo Z do estágio da amostra|
|TEST:MOTORS:m4|Motor|Eixo de rotação em X do estágio da amostra|
|TEST:MOTORS:m5|Motor|Eixo de rotação em Y do estágio da amostra|
|TEST:MOTORS:m6|Motor|Eixo de rotação em Z do estágio da amostra|
|TEST:MOTORS:m7|Motor|Eixo de translação auxiliar|
|TEST:MOTORS:m8|Motor|Eixo de translação auxiliar|

#### Implementação inicial

In [ ]:
@timed_function(print_timing=True)
def my_scan():
    import numpy as np
    from epics import caget, caput

    my_data = {}

    caput("TEST:MOTORS:m2.CNEN", 1, wait=True)
    for pos in np.linspace(-20, 20, 5):
        caput("TEST:MOTORS:m1", pos, wait=True)

        caput("TEST:DETECTOR:Trigger", 1)
        det_data  = caget("TEST:DETECTOR:Data")
        my_data[pos] = det_data

    caput("TEST:MOTORS:m2.CNEN", 0, wait=True)

    print(my_data)

my_scan()

----
Fix `TEST:MOTORS:m1` -> `TEST:MOTORS:m2`

Fix `TEST:DETECTOR:Data` -> `TEST:DETECTOR:Data_RBV`

----

In [ ]:
@timed_function(print_timing=True)
def my_scan():
    import numpy as np
    from epics import caget, caput

    my_data = {}

    caput("TEST:MOTORS:m2.CNEN", 1, wait=True)
    for pos in np.linspace(-20, 20, 5):
        caput("TEST:MOTORS:m2", pos, wait=True)

        caput("TEST:DETECTOR:Trigger", 1)
        det_data  = caget("TEST:DETECTOR:Data_RBV")
        my_data[pos] = det_data

    caput("TEST:MOTORS:m2.CNEN", 0, wait=True)

    print(my_data)

my_scan()

----
Fix `Trigger` / `AcquisitionNumber`

----

In [ ]:
@timed_function(print_timing=True)
def my_scan():
    import time
    import numpy as np
    from epics import caget, caput

    my_data = {}

    caput("TEST:MOTORS:m2.CNEN", 1, wait=True)
    while caget("TEST:MOTORS:m2.CNEN") != 1:
        time.sleep(0.1)

    for pos in np.linspace(-20, 20, 5):
        caput("TEST:MOTORS:m2", pos, wait=True)

        while caget("TEST:DETECTOR:Trigger_RBV") != 0:
            time.sleep(0.1)

        acq_num = caget("TEST:DETECTOR:AcquisitionNumber_RBV")
        caput("TEST:DETECTOR:Trigger", 1)
        while caget("TEST:DETECTOR:AcquisitionNumber_RBV") == acq_num:
            time.sleep(0.1)

        det_data  = caget("TEST:DETECTOR:Data_RBV")
        my_data[pos] = det_data

    caput("TEST:MOTORS:m2.CNEN", 0, wait=True)

    print(my_data)

my_scan()

----
Factor out PV prefixes

----

In [ ]:
@timed_function(print_timing=True)
def my_scan():
    import time
    import numpy as np
    from epics import caget, caput

    Y_MOTOR_PREFIX = "TEST:MOTORS:m2"
    Y_MOTOR_ENABLE = Y_MOTOR_PREFIX + ".CNEN"
    DETECTOR_PREFIX = "TEST:DETECTOR"
    DETECTOR_TRIGGER = DETECTOR_PREFIX + ":Trigger"
    DETECTOR_TRIGGER_RBV = DETECTOR_PREFIX + ":Trigger_RBV"
    DETECTOR_ACQ_NUMBER = DETECTOR_PREFIX + ":AcquisitionNumber_RBV"
    DETECTOR_DATA = DETECTOR_PREFIX + ":Data_RBV"

    my_data = {}

    caput(Y_MOTOR_ENABLE, 1, wait=True)
    while caget(Y_MOTOR_ENABLE) != 1:
        time.sleep(0.1)

    for pos in np.linspace(-20, 20, 5):
        caput(Y_MOTOR_PREFIX, pos, wait=True)

        while caget(DETECTOR_TRIGGER_RBV) != 0:
            time.sleep(0.1)

        acq_num = caget(DETECTOR_ACQ_NUMBER)
        caput(DETECTOR_TRIGGER, 1)
        while caget(DETECTOR_ACQ_NUMBER) == acq_num:
            time.sleep(0.1)

        det_data  = caget(DETECTOR_DATA)
        my_data[pos] = det_data

    caput(Y_MOTOR_ENABLE, 0, wait=True)

    print(my_data)

my_scan()

----
Live plot

----

In [ ]:
%matplotlib widget

@timed_function(print_timing=True)
def my_scan():
    import time
    import numpy as np
    from epics import caget, caput

    from IPython.display import display, clear_output
    import matplotlib.pyplot as plt

    Y_MOTOR_PREFIX = "TEST:MOTORS:m2"
    Y_MOTOR_ENABLE = Y_MOTOR_PREFIX + ".CNEN"
    DETECTOR_PREFIX = "TEST:DETECTOR"
    DETECTOR_TRIGGER = DETECTOR_PREFIX + ":Trigger"
    DETECTOR_TRIGGER_RBV = DETECTOR_PREFIX + ":Trigger_RBV"
    DETECTOR_ACQ_NUMBER = DETECTOR_PREFIX + ":AcquisitionNumber_RBV"
    DETECTOR_DATA = DETECTOR_PREFIX + ":Data_RBV"

    plt.ion()
    figure, ax = plt.subplots()

    def update_plot(data: dict):
        X = list(data.keys())
        Y = list(data.values())

        clear_output(True)
        ax.clear()

        if len(X) > 1:
            slope, intercept = np.polyfit(X, Y, deg=1)
            ax.errorbar(X, [slope*i + intercept for i in X], yerr=1, color='r', label=f"Fitting ({slope:.4f}x + {intercept:.4f})")
        ax.plot(X, Y, zorder=50, marker='o')
        figure.gca().relim()
        figure.gca().autoscale_view()
        figure.legend(framealpha=1.0)
        display(figure)


    my_data = {}

    caput(Y_MOTOR_ENABLE, 1, wait=True)
    while caget(Y_MOTOR_ENABLE) != 1:
        time.sleep(0.1)

    for pos in np.linspace(-20, 20, 5):
        caput(Y_MOTOR_PREFIX, pos, wait=True)

        while caget(DETECTOR_TRIGGER_RBV) != 0:
            time.sleep(0.1)

        acq_num = caget(DETECTOR_ACQ_NUMBER)
        caput(DETECTOR_TRIGGER, 1)
        while caget(DETECTOR_ACQ_NUMBER) == acq_num:
            time.sleep(0.1)

        det_data  = caget(DETECTOR_DATA)
        my_data[pos] = det_data

        update_plot(my_data)

    caput(Y_MOTOR_ENABLE, 0, wait=True)

my_scan()

----
Metadata reading

----

In [ ]:
%matplotlib widget

@timed_function(print_timing=True)
def my_scan():
    import time
    import numpy as np
    from epics import caget, caput

    from IPython.display import display, clear_output
    import matplotlib.pyplot as plt

    Y_MOTOR_PREFIX = "TEST:MOTORS:m2"
    Y_MOTOR_ENABLE = Y_MOTOR_PREFIX + ".CNEN"
    DETECTOR_PREFIX = "TEST:DETECTOR"
    DETECTOR_TRIGGER = DETECTOR_PREFIX + ":Trigger"
    DETECTOR_TRIGGER_RBV = DETECTOR_PREFIX + ":Trigger_RBV"
    DETECTOR_ACQ_NUMBER = DETECTOR_PREFIX + ":AcquisitionNumber_RBV"
    DETECTOR_DATA = DETECTOR_PREFIX + ":Data_RBV"

    plt.ion()
    figure, ax = plt.subplots()

    def update_plot(data: dict):
        X = list(data.keys())
        Y = list(data.values())

        clear_output(True)
        ax.clear()

        if len(X) > 1:
            slope, intercept = np.polyfit(X, Y, deg=1)
            ax.errorbar(X, [slope*i + intercept for i in X], yerr=1, color='r', label=f"Fitting ({slope:.4f}x + {intercept:.4f})")
        ax.plot(X, Y, zorder=50, marker='o')
        figure.gca().relim()
        figure.gca().autoscale_view()
        figure.legend(framealpha=1.0)
        display(figure)

    def retrieve_metadata(prefix: str = ""):
        md = dict()
        for suffix, name in zip(["m1", "m2", "m3", "m4", "m5", "m6", "m7", "m8"], ["x", "y", "z", "Rx", "Ry", "Rz", "Utg", "Uth"]):
            md[f"{prefix}{name}_loc"] = caget(f"TEST:MOTORS:{suffix}.RBV", use_monitor=False)
            md[f"{prefix}{name}_step_size"] = caget(f"TEST:MOTORS:{suffix}.MRES", use_monitor=False)
            md[f"{prefix}{name}_velocity"] = caget(f"TEST:MOTORS:{suffix}.VELO", use_monitor=False)
            md[f"{prefix}{name}_base_velocity"] = caget(f"TEST:MOTORS:{suffix}.VBAS", use_monitor=False)
            md[f"{prefix}{name}_unit"] = caget(f"TEST:MOTORS:{suffix}.EGU", use_monitor=False)
            md[f"{prefix}{name}_display_precision"] = caget(f"TEST:MOTORS:{suffix}.PREC", use_monitor=False)
            md[f"{prefix}{name}_version"] = caget(f"TEST:MOTORS:{suffix}.VERS", use_monitor=False)
            md[f"{prefix}{name}_enabled"] = caget(f"TEST:MOTORS:{suffix}.CNEN", use_monitor=False)
        return md

    my_data = {}
    my_metadata = retrieve_metadata("before")

    caput(Y_MOTOR_ENABLE, 1, wait=True)
    while caget(Y_MOTOR_ENABLE) != 1:
        time.sleep(0.1)

    for pos in np.linspace(-20, 20, 5):
        caput(Y_MOTOR_PREFIX, pos, wait=True)

        while caget(DETECTOR_TRIGGER_RBV) != 0:
            time.sleep(0.1)

        acq_num = caget(DETECTOR_ACQ_NUMBER)
        caput(DETECTOR_TRIGGER, 1)
        while caget(DETECTOR_ACQ_NUMBER) == acq_num:
            time.sleep(0.1)

        det_data  = caget(DETECTOR_DATA)
        my_data[pos] = det_data

        update_plot(my_data)

    caput(Y_MOTOR_ENABLE, 0, wait=True)

    my_metadata.update(retrieve_metadata("after"))

my_scan()

In [ ]:
%matplotlib widget

@timed_function(print_timing=True)
def my_scan():
    import time
    import numpy as np
    from epics import caget, caput, get_pv

    from IPython.display import display, clear_output
    import matplotlib.pyplot as plt

    Y_MOTOR_PREFIX = "TEST:MOTORS:m2"
    Y_MOTOR_ENABLE = Y_MOTOR_PREFIX + ".CNEN"
    DETECTOR_PREFIX = "TEST:DETECTOR"
    DETECTOR_TRIGGER = DETECTOR_PREFIX + ":Trigger"
    DETECTOR_TRIGGER_RBV = DETECTOR_PREFIX + ":Trigger_RBV"
    DETECTOR_ACQ_NUMBER = DETECTOR_PREFIX + ":AcquisitionNumber_RBV"
    DETECTOR_DATA = DETECTOR_PREFIX + ":Data_RBV"

    plt.ion()
    figure, ax = plt.subplots()

    def update_plot(data: dict):
        X = list(data.keys())
        Y = list(data.values())

        clear_output(True)
        ax.clear()

        if len(X) > 1:
            slope, intercept = np.polyfit(X, Y, deg=1)
            ax.errorbar(X, [slope*i + intercept for i in X], yerr=1, color='r', label=f"Fitting ({slope:.4f}x + {intercept:.4f})")
        ax.plot(X, Y, zorder=50, marker='o')
        figure.gca().relim()
        figure.gca().autoscale_view()
        figure.legend(framealpha=1.0)
        display(figure)

    def retrieve_metadata(prefix: str = ""):
        md = dict()
        pvs = dict()
        for suffix, name in zip(["m1", "m2", "m3", "m4", "m5", "m6", "m7", "m8"], ["x", "y", "z", "Rx", "Ry", "Rz", "Utg", "Uth"]):
            pvs[f"{prefix}{name}_loc"] = get_pv(f"TEST:MOTORS:{suffix}.RBV", use_monitor=False)
            pvs[f"{prefix}{name}_step_size"] = get_pv(f"TEST:MOTORS:{suffix}.MRES", use_monitor=False)
            pvs[f"{prefix}{name}_velocity"] = get_pv(f"TEST:MOTORS:{suffix}.VELO", use_monitor=False)
            pvs[f"{prefix}{name}_base_velocity"] = get_pv(f"TEST:MOTORS:{suffix}.VBAS", use_monitor=False)
            pvs[f"{prefix}{name}_unit"] = get_pv(f"TEST:MOTORS:{suffix}.EGU", use_monitor=False)
            pvs[f"{prefix}{name}_display_precision"] = get_pv(f"TEST:MOTORS:{suffix}.PREC", use_monitor=False)
            pvs[f"{prefix}{name}_version"] = get_pv(f"TEST:MOTORS:{suffix}.VERS", use_monitor=False)
            pvs[f"{prefix}{name}_enabled"] = get_pv(f"TEST:MOTORS:{suffix}.CNEN", use_monitor=False)
        for pv in pvs.values():
            pv.wait_for_connection(timeout=1.0)
        for name, pv in pvs.items():
            if pv.isConnected():
                md[name] = pv.get()
        return md

    my_data = {}
    my_metadata = retrieve_metadata("before")

    caput(Y_MOTOR_ENABLE, 1, wait=True)
    while caget(Y_MOTOR_ENABLE) != 1:
        time.sleep(0.1)

    for pos in np.linspace(-20, 20, 5):
        caput(Y_MOTOR_PREFIX, pos, wait=True)

        while caget(DETECTOR_TRIGGER_RBV) != 0:
            time.sleep(0.1)

        acq_num = caget(DETECTOR_ACQ_NUMBER)
        caput(DETECTOR_TRIGGER, 1)
        while caget(DETECTOR_ACQ_NUMBER) == acq_num:
            time.sleep(0.1)

        det_data  = caget(DETECTOR_DATA)
        my_data[pos] = det_data

        update_plot(my_data)

    caput(Y_MOTOR_ENABLE, 0, wait=True)

    my_metadata.update(retrieve_metadata("after"))

my_scan()

### Abstração de hardware: `ophyd`